tests to use @overloads in a stub package 

Version 
Requirements:
 - [x] create board stubs of same version 
    - [x] samd seeed_wio_terminal
 - [x] create docstubs
 - [ ] Integrate params and @overloads from reference stubs
    - [ ] Need additional fixup 
 - [x] merge with board stubs 
 - [x] create stub package port
 - [x] create stub package port + board

 
FIXME: 
  - [ ] Need additional cleanup after building a package 
  - [ ] add _mpy_shed to micropython-stdlib-stubs package or as a independent package micropython-typeshed ?
  - [ ] aadd _rp2 module to all builds of the rp2 port 


    
- [ ] run test 
- [ ] manual inspection on resolving the stubs



In [1]:
# set parameters
from pathlib import Path

version = "v1.24.0"
flat_version = version.replace(".", "_")

mps_p = Path("./repos/micropython-stubs")
# Destination
reference_p = mps_p / "micropython-reference"

docstubs_p = mps_p / f"stubs/micropython-{flat_version}-docstubs"

In [2]:
! stubber docstubs --version {version}

18:04:22 | ℹ️ micropython-stubber 1.23.2a0
18:04:22 | ℹ️ fetch updates
18:04:24 | ℹ️ Switching to v1.24.0
18:04:31 | ℹ️ repos\micropython                        v1.24.0-dirty
18:04:31 | ℹ️ repos\micropython-lib                    v1.24.0
18:04:31 | ℹ️ repos\micropython\lib\micropython-lib    v1.24.0
18:04:31 | ℹ️ Get docstubs for MicroPython v1.24.0
18:04:31 | ℹ️  - Writing to: 
repos\micropython-stubs\stubs\micropython-v1_24_0-docstubs\array\__init__.pyi
18:04:31 | ℹ️  - Writing to: 
repos\micropython-stubs\stubs\micropython-v1_24_0-docstubs\asyncio\__init__.pyi
18:04:31 | ℹ️  - Writing to: 
repos\micropython-stubs\stubs\micropython-v1_24_0-docstubs\binascii\__init__.py
i
18:04:31 | ℹ️  - Writing to: 
repos\micropython-stubs\stubs\micropython-v1_24_0-docstubs\bluetooth\__init__.p
yi
18:04:31 | ℹ️  - Writing to: 
repos\micropython-stubs\stubs\micropython-v1_24_0-docstubs\btree\__init__.pyi
18:04:31 | ℹ️  - Writing to: 
repos\micropython-stubs\stubs\micropython-v1_24_0-docstubs\cmath\__

In [3]:
# Integrate params and @overloads from reference stubs into the doc-stubs 
! stubber enrich --params-only --stubs {docstubs_p} --docstubs {reference_p}

Traceback (most recent call last):
  File "D:\mypython\micropython-stubber\.venv\Lib\site-packages\libcst\codemod\_runner.py", line 143, in transform_module
    output_tree = transformer.transform_module(input_tree)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\mypython\micropython-stubber\.venv\Lib\site-packages\libcst\codemod\_command.py", line 87, in transform_module
    tree = self._instantiate_and_run(transform, tree)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\mypython\micropython-stubber\.venv\Lib\site-packages\libcst\codemod\_command.py", line 57, in _instantiate_and_run
    return inst.transform_module(tree)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\mypython\micropython-stubber\.venv\Lib\site-packages\libcst\codemod\_codemod.py", line 108, in transform_module
    return self.transform_module_impl(tree_with_metadata)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\mypython\micropython-stubber\.venv\Lib\si

18:04:43 | ℹ️ micropython-stubber 1.23.2a0
18:04:43 | ℹ️ Enriching 
repos\micropython-stubs\stubs\micropython-v1_24_0-docstubs with 
repos\micropython-stubs\micropython-reference
18:04:43 | ℹ️ Enrich folder 
repos\micropython-stubs\stubs\micropython-v1_24_0-docstubs.
18:04:43 | ℹ️ Merge 
repos\micropython-stubs\stubs\micropython-v1_24_0-docstubs\_thread\__init__.pyi
from repos\micropython-stubs\micropython-reference\_thread\__init__.pyi
18:04:43 | ℹ️ Merge 
repos\micropython-stubs\stubs\micropython-v1_24_0-docstubs\array\__init__.pyi 
from repos\micropython-stubs\micropython-reference\uarray.pyi
18:04:43 | ℹ️ add: from array import *
18:04:43 | ℹ️ Merge 
repos\micropython-stubs\stubs\micropython-v1_24_0-docstubs\array\__init__.pyi 
from repos\micropython-stubs\micropython-reference\array\__init__.pyi
18:04:43 | ℹ️ Change __getitem__ to @overload
18:04:43 | ℹ️ Change __setitem__ to @overload
18:04:43 | ℹ️ Merge 
repos\micropython-stubs\stubs\micropython-v1_24_0-docstubs\asyncio\__init__

In [4]:
### sanity check on formatting
! ruff check {docstubs_p} --ignore F4 --ignore UP0 --ignore F811 

All checks passed!


In [13]:
# remove the DMA, Flash , PIO and StateMachine classes from rp2 as they are in _rp2.**

for stem in ["DMA", "Flash", "PIO", "StateMachine"]:
    file = docstubs_p / f"rp2/{stem}.pyi"
    if file.exists():
        file.unlink()
        print(f"Removed {file}")

In [6]:
# clean up duplicate typevar definitions

rp2_comment_hack = [
    "from _rp2.DMA import DMA as DMA",
    "from _rp2.Flash import Flash as Flash",
    "from _rp2.PIO import PIO as PIO",
    "from _rp2.StateMachine import StateMachine as StateMachine",
]


def remove_duplicates(file):
    with open(file, encoding="utf-8") as f:
        lines = f.readlines()

    typevars = {}
    comments = {}
    with open(file, "w", encoding="utf-8") as f:
        for line in lines:
            if line.startswith("# MCU:") or line.startswith("# Stubber"):
                if line in comments:
                    continue
                comments[line] = True
            # Remove AnyReadableBuf and AnyWritableBuf from io.pyi and uio.pyi
            if file.name in ["io.pyi", "uio.pyi"]:
                if line.startswith("AnyReadableBuf = TypeVar") or line.startswith(
                    "AnyWritableBuf = TypeVar"
                ):
                    continue
            if file.name in ["rp2.pyi"]:
                if line.strip() in rp2_comment_hack:
                    line = f"# {line}"
            # Ugly hack to fix TypeVar imported from typing_extensions AND typing
            if line == "from typing_extensions import TypeVar\n":
                line = "from typing import TypeVar\n"
            elif ": TypeVar" in line:
                if line in typevars:
                    continue
                typevars[line] = True
            f.write(line)


for file in docstubs_p.rglob("*.pyi"):
    remove_duplicates(file)

In [7]:
# Clean formatting and upgrade typing notation to 3.10
 
! ruff check {docstubs_p} --fix --unsafe-fixes --ignore UP006 --ignore  UP035 --ignore F405

# to fix in the u* modules 
# [ ] different imports of typevar
# [ ] Duplicate defenitions of AnyWritableBuffer

Found 4 errors (4 fixed, 0 remaining).


In [8]:
# run autoflake on all files - one by one to get the __init__.pyi files
for f in docstubs_p.rglob("*.pyi"):
    ! autoflake {f} --in --imports typing_extensions,_mpy_shed,_typeshed

! isort {docstubs_p}

Fixing D:\mypython\micropython-stubber\repos\micropython-stubs\stubs\micropython-v1_24_0-docstubs\uarray.pyi
Fixing D:\mypython\micropython-stubber\repos\micropython-stubs\stubs\micropython-v1_24_0-docstubs\ubinascii.pyi
Fixing D:\mypython\micropython-stubber\repos\micropython-stubs\stubs\micropython-v1_24_0-docstubs\uio.pyi
Fixing D:\mypython\micropython-stubber\repos\micropython-stubs\stubs\micropython-v1_24_0-docstubs\ujson.pyi
Fixing D:\mypython\micropython-stubber\repos\micropython-stubs\stubs\micropython-v1_24_0-docstubs\uos.pyi
Fixing D:\mypython\micropython-stubber\repos\micropython-stubs\stubs\micropython-v1_24_0-docstubs\uselect.pyi
Fixing D:\mypython\micropython-stubber\repos\micropython-stubs\stubs\micropython-v1_24_0-docstubs\usocket.pyi
Fixing D:\mypython\micropython-stubber\repos\micropython-stubs\stubs\micropython-v1_24_0-docstubs\ussl.pyi
Fixing D:\mypython\micropython-stubber\repos\micropython-stubs\stubs\micropython-v1_24_0-docstubs\ustruct.pyi
Fixing D:\mypython\mic

In [ ]:
# ! stubber build --version {version} --port "samd" --board "seeed_wio_terminal" --board "GENERIC"

20:07:34 | ℹ️ micropython-stubber 1.23.2a0
20:07:34 | ℹ️ Build micropython ['v1.24.0'] ['samd'] ['seeed_wio_terminal', 
'GENERIC']
20:07:34 | ℹ️ checking 2 possible board candidates
20:07:34 | ℹ️ Build: micropython-v1_24_0-samd-stubs
20:07:34 | ℹ️ - Update micropython-v1_24_0-samd-stubs
20:07:45 | ℹ️ Found changes to package sources: micropython-samd-stubs 
1.24.0.post1 
20:07:48 | ℹ️ Build: micropython-v1_24_0-samd-seeed_wio_terminal-stubs
20:07:48 | ℹ️ - Update micropython-v1_24_0-samd-seeed_wio_terminal-stubs
20:07:50 | ℹ️ Found changes to package sources: 
micropython-samd-seeed_wio_terminal-stubs 1.24.0.post1 
20:07:52 | ℹ️ Built 2 stub packages
result    name                                       version       error    path
--------  -----------------------------------------  ------------  -------  ---------------------------------------------------------------------------------
Build OK  micropython-samd-stubs                     1.24.0.post1           repos/micropython-stubs/pu

In [ ]:
# ### sanity check on formatting
# stub_p = mps_p /  f"publish/micropython-{flat_version}-samd-seeed_wio_terminal-stubs"
# ! ruff check {stub_p} --ignore F4 --ignore UP0 --ignore F811 

All checks passed!


In [35]:
! stubber -VV build --version {version} --port "rp2" --board "RPI_PICO" --board "GENERIC" --board "RPI_PICO_W"

22:56:39 | ℹ️ micropython-stubber 1.23.2a0
22:56:39 | ℹ️ Log level set to TRACE
22:56:39 | ℹ️ Build micropython ['v1.24.0'] ['rp2'] ['RPI_PICO', 'GENERIC', 
'RPI_PICO_W']
22:56:39 | ℹ️ checking 3 possible board candidates
22:56:39 | 🐞 No package found for micropython-rp2-stubs in database, creating 
new package
22:56:39 | ℹ️ Build: micropython-v1_24_0-rp2-stubs
22:56:39 | ℹ️ - Update micropython-v1_24_0-rp2-stubs
22:56:39 | ✏️ repos/micropython-stubs/publish/micropython-v1_24_0-rp2-stubs
22:56:39 | 🐞 Copying Merged stubs from micropython-v1_24_0-rp2-RPI_PICO-merged
22:56:39 | 🐞 Copying Core stubs from micropython-core
22:56:39 | 🐞 poetry ['check', '-vvv'] starting
22:56:41 | ✏️ poetry ['check', '-vvv'] completed
22:56:41 | ✏️ changed = True | Stored: None | Current: 
c936a20bfb12d9990748f282d07c66014a3867bc
22:56:41 | ℹ️ Found changes to package sources: micropython-rp2-stubs 
1.24.0.post1 
22:56:41 | ✏️ Old hash None != New hash c936a20bfb12d9990748f282d07c66014a3867bc
22:56:41 | 🐞 mi

In [36]:
# ! pytest repos/micropython-stubs/tests/quality_tests/test_snippets.py::test_typecheck -k "local-v1.24.0-preview-rp2 and -pyright"
port = "rp2"
# ! pytest repos/micropython-stubs/tests/quality_tests/test_snippets.py::test_typecheck -k "local-v1.24.0-preview-{port}- and -pyright"
! pytest repos/micropython-stubs/tests/quality_tests/test_snippets.py::test_typecheck -k "local-v1.24.0-{port}- and -pyright"

============================= test session starts =============================
platform win32 -- Python 3.11.9, pytest-7.4.4, pluggy-1.5.0 -- D:\mypython\micropython-stubber\.venv\Scripts\python.exe
cachedir: .pytest_cache
hypothesis profile 'default' -> database=DirectoryBasedExampleDatabase(WindowsPath('d:/mypython/micropython-stubber/.hypothesis/examples'))
metadata: {'Python': '3.11.9', 'Platform': 'Windows-10-10.0.26100-SP0', 'Packages': {'pytest': '7.4.4', 'pluggy': '1.5.0'}, 'Plugins': {'hypothesis': '6.108.10', 'github-actions-annotate-failures': '0.2.0', 'json-report': '1.5.0', 'metadata': '3.1.1', 'mock': '3.14.0'}}
rootdir: d:\mypython\micropython-stubber\repos\micropython-stubs
configfile: pyproject.toml
plugins: hypothesis-6.108.10, github-actions-annotate-failures-0.2.0, json-report-1.5.0, metadata-3.1.1, mock-3.14.0
collecting ... collected 628 items / 605 deselected / 23 selected

repos\micropython-stubs\tests\quality_tests\test_snippets.py::test_typecheck[local-v1.24.

2024-11-28 22:57:18.959 | DEBUG    | conftest:type_stub_cache_path:98 - setup install type_stubs to cache: local, v1.24.0, rp2
2024-11-28 22:57:18.970 | DEBUG    | conftest:type_stub_cache_path:114 - Using cached type stubs for rp2 v1.24.0
2024-11-28 22:57:34.664 | DEBUG    | conftest:type_stub_cache_path:98 - setup install type_stubs to cache: local, v1.24.0, rp2-rpi_pico
2024-11-28 22:57:34.664 | DEBUG    | conftest:type_stub_cache_path:114 - Using cached type stubs for rp2-rpi_pico v1.24.0
2024-11-28 22:57:49.847 | DEBUG    | conftest:type_stub_cache_path:98 - setup install type_stubs to cache: local, v1.24.0, rp2-rpi_pico_w
2024-11-28 22:57:49.849 | DEBUG    | conftest:type_stub_cache_path:114 - Using cached type stubs for rp2-rpi_pico_w v1.24.0
2024-11-28 22:58:09.520 | DEBUG    | conftest:type_stub_cache_path:98 - setup install type_stubs to cache: local, v1.24.0, rp2-rpi_pico2
2024-11-28 22:58:09.522 | DEBUG    | conftest:type_stub_cache_path:114 - Using cached type stubs for rp